In [117]:
import random
import pandas as pd
import numpy as np
import os
from glob import glob
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler


In [118]:
def make_dataset(all_input_list, all_target_list):
    '''
    Train, Test데이터를 하나의 데이터 프레임으로 변경
    '''
    df_all = pd.DataFrame()
    length = len(all_input_list)
    for idx in range(length):
        X = pd.read_csv(all_input_list[idx])
        y = pd.read_csv(all_target_list[idx])
        y['DAT'] = y['DAT']-1
        df_concat = pd.merge(X, y, on='DAT', how='left')
        df_concat['Case'] = idx+1
        df_all = pd.concat([df_all, df_concat])
    return df_all


def time_value(df):
    ''' 
    ex) 00:59:59 => 01:00:00으로 변환 후 시간단위만 추출
    '''
    df['obs_time'] = pd.to_datetime(df["obs_time"]) + datetime.timedelta(seconds=1)
    df['obs_time'] = df['obs_time'].dt.hour
    return df

def limit_range(df):
    '''
    환경 변수 별 제한 범위를 넘어서는 값을 결측치 처리
    '''
    df.loc[(df['내부온도관측치'] < 4) | (df['내부온도관측치'] > 40), '내부온도관측치'] = np.nan
    df.loc[(df['내부습도관측치'] < 0) | (df['내부습도관측치'] > 100), '내부습도관측치'] = np.nan
    df.loc[(df['co2관측치'] < 0) | (df['co2관측치'] > 1200), 'co2관측치'] = np.nan
    df.loc[(df['ec관측치'] < 0) | (df['ec관측치'] > 8), 'ec관측치'] = np.nan
    df.loc[(df['시간당분무량'] < 0) | (df['시간당분무량'] > 3000), '시간당분무량'] = np.nan
    df.loc[(df['일간누적분무량'] < 0) | (df['일간누적분무량'] > 72000), '일간누적분무량'] = np.nan
    df.loc[(df['시간당백색광량'] < 0) | (df['시간당백색광량'] > 120000), '시간당백색광량'] = np.nan
    df.loc[(df['일간누적백색광량'] < 0) | (df['일간누적백색광량'] > 2880000), '일간누적백색광량'] = np.nan
    df.loc[(df['시간당적색광량'] < 0) | (df['시간당적색광량'] > 120000), '시간당적색광량'] = np.nan
    df.loc[(df['일간누적적색광량'] < 0) | (df['일간누적적색광량'] > 2880000), '일간누적적색광량'] = np.nan
    df.loc[(df['시간당청색광량'] < 0) | (df['시간당청색광량'] > 120000), '시간당청색광량'] = np.nan
    df.loc[(df['일간누적청색광량'] < 0) | (df['일간누적청색광량'] > 2880000), '일간누적청색광량'] = np.nan
    df.loc[(df['시간당총광량'] < 0) | (df['시간당총광량'] > 120000), '시간당총광량'] = np.nan
    df.loc[(df['일간누적총광량'] < 0) | (df['일간누적총광량'] > 2880000), '일간누적총광량'] = np.nan
    return df

def col_cumsum(df, col, cum_col):
    '''
    시간값에 이상치가 있어서 누적값을 새로 생성
    '''
    import itertools
    df[cum_col] = 0
    for i in range(784):
        result = itertools.accumulate(df[col][i*24:(i+1)*24])
        cumsum = [value for value in result]
        df[cum_col][i*24:(i+1)*24] = cumsum
        
    return df


def cumsum_group_max(df):
    '''
    *누적값은 media이 아닌 일별 최대값이 들어가야함
    '''
    df_a = pd.DataFrame()

    for i, v in enumerate(df["Case"].unique()):
        train_old = df[df['Case']==v]
        train_old = train_old.groupby(['DAT']).max().reset_index()
        df_a = pd.concat([df_a, train_old])
    return df_a


def group_median(df):  
    '''
    변수의 Case별로 DAT을 기준으로 groupby 하여 대표값 설정
    '''
    df_a = pd.DataFrame()

    for i, v in enumerate(df["Case"].unique()):
        train_old = df[df['Case']==v]
        train_old = train_old.groupby(['DAT']).mean().reset_index()
        df_a = pd.concat([df_a, train_old])
    return df_a


def concat_df(df, cumsum_df):
    '''
    누적값을 원래 데이터와 concat
    '''
    cumsum_list = ['일간누적분무량', '일간누적백색광량', '일간누적적색광량', '일간누적청색광량', '일간누적총광량']
    for col in cumsum_list:
        df[col] = cumsum_df[col]
        
    return df

In [119]:
train_input_list = sorted(glob('./data/train_input/*.csv'))
train_target_list = sorted(glob('./data/train_target/*.csv'))

test_input_list = sorted(glob('./data/test_input/*.csv'))
test_target_list = sorted(glob('./data/test_target/*.csv'))

In [120]:
train = make_dataset(train_input_list, train_target_list)
test = make_dataset(test_input_list, test_target_list)

train = time_value(train)
test = time_value(test)

train = limit_range(train)
test = limit_range(test)

train = train.fillna(method='ffill')
test = test.fillna(method='ffill')

test['predicted_weight_g'] = 0

In [121]:
train = col_cumsum(train, "시간당분무량", "일간누적분무량")
train = col_cumsum(train, "시간당백색광량", "일간누적백색광량")
train = col_cumsum(train, "시간당적색광량", "일간누적적색광량")
train = col_cumsum(train, "시간당청색광량", "일간누적청색광량")
train = col_cumsum(train, "시간당총광량", "일간누적총광량")

C:\Users\kki96\AppData\Local\Temp\ipykernel_21400\2452025495.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[cum_col][i*24:(i+1)*24] = cumsum


In [122]:
def diff_temp(train, test):
    '''
    일교차
    '''
    train_diff_temp = []
    for i in range(784):
        temp_train = train[i*24:(i+1)*24]
        diff_train = temp_train['내부온도관측치'].max() - temp_train['내부온도관측치'].min()
        train_diff_temp.append(diff_train)

    test_diff_temp = []
    for i in range(140):
        temp_test = test[i*24:(i+1)*24]
        diff_test = temp_test['내부온도관측치'].max() - temp_test['내부온도관측치'].min()
        test_diff_temp.append(diff_test)

    return train_diff_temp, test_diff_temp


def none_light(train, test):
    '''
    광합성을 못하는 시간
    '''
    train_night = []
    for i in range(784):
        nigth_train = train[i*24:(i+1)*24]
        train_night.append(nigth_train[nigth_train['시간당백색광량']==0]['시간당백색광량'].count())
    
    test_night = []
    for i in range(140):
        nigth_test = test[i*24:(i+1)*24]
        test_night.append(nigth_test[nigth_test['시간당백색광량']==0]['시간당백색광량'].count())
    
    return train_night, test_night


def water(train, test):
    '''
    하루에 물을 주는 횟수
    '''
    train_water = []
    for i in range(784):
        water_train = train[i*24:(i+1)*24]
        train_water.append(water_train[water_train['시간당분무량']!=0]['시간당분무량'].count())
    
    test_water = []
    for i in range(140):
        water_test = test[i*24:(i+1)*24]
        test_water.append(water_test[water_test['시간당분무량']!=0]['시간당분무량'].count())
    
    return train_water, test_water

In [123]:
train_temp, test_temp = diff_temp(train, test)
train_dark, test_dark = none_light(train, test)
train_water, test_water = water(train, test)

In [150]:
df = train.copy()

In [151]:
def time_split(df):
    df.loc[(df['obs_time'] < 6), '6time'] = 1
    df.loc[(df['obs_time'] >=6) & (df['obs_time'] < 12), '6time'] = 2
    df.loc[(df['obs_time'] >= 12) & (df['obs_time'] < 19), '6time'] = 3
    df.loc[(df['obs_time'] >= 19) & (df['obs_time'] <= 24), '6time'] = 4
    
    return df

In [152]:
df = time_split(df)

In [153]:
def pivot_data(df):
    '''
    6시간 단위의 pivot table 생성
    '''
    df = pd.pivot_table(df, index=['DAT', 'Case'], columns=['6time'], aggfunc='mean')
    df.columns = [''.join(str(col)) for col in df.columns]
    df = df.reset_index()
    
    return df

In [26]:
train_cumsum = cumsum_group_max(train_cumsum)
test_cumsum = cumsum_group_max(test_cumsum)

train_rep = group_median(train)
test_rep = group_median(test)

train = concat_df(train_rep, train_cumsum)
test = concat_df(test_rep, test_cumsum)

train['diff_temp'] = train_temp
test['diff_temp'] = test_temp
train['dark'] = train_dark
test['dark'] = test_dark
train['water'] = train_water
test['water'] = test_water

In [27]:
df_a = train.drop(['obs_time'], axis=1)
df_b = test.drop(['obs_time'], axis=1)

In [28]:
train = df_a.copy()
test = df_b.copy()

In [29]:
def accumulate(train, test, col):
    
    train['월간'+col] = 0
    for i in range(28):
        result = (train['일간'+col][i*28:(i+1)*28].cumsum())
        train['월간'+col][i*28:(i+1)*28] = result


    test['월간'+col] = 0
    for i in range(5):
        result = (test['일간'+col][i*28:(i+1)*28].cumsum())
        test["월간"+col][i*28:(i+1)*28] = result
        
accumulate(train, test, '누적분무량')
accumulate(train, test, '누적백색광량')
accumulate(train, test, '누적적색광량')
accumulate(train, test, '누적청색광량')
accumulate(train, test, '누적총광량')

C:\Users\kki96\AppData\Local\Temp\ipykernel_21400\3508050368.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['월간'+col][i*28:(i+1)*28] = result
C:\Users\kki96\AppData\Local\Temp\ipykernel_21400\3508050368.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["월간"+col][i*28:(i+1)*28] = result
C:\Users\kki96\AppData\Local\Temp\ipykernel_21400\3508050368.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['월간'+col][i*28

In [13]:
# train = train.reset_index()
# test = test.reset_index()
# train_new = pd.read_csv('train_col.csv')
# test_new = pd.read_csv('test_col.csv')

# train = pd.concat([train, train_new], axis=1)
# test = pd.concat([test, test_new], axis=1)

In [14]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import glob
def model_train(train, test, model):
    '''
    train shape = (784, 18), test shape = (140, 18)
    model => sklearn api model
    '''

    X = train.drop(['predicted_weight_g', 'Case', 'index'], axis=1)
    y = train['predicted_weight_g']

    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=113, shuffle=True)
    
    x_test = test.drop(['predicted_weight_g', 'Case', 'index'], axis=1)
    
    model.fit(X_train, y_train)
    y_val_pred = model.predict(X_val)

    rmse = mean_squared_error(y_val, y_val_pred)**0.5
    print(f"validation rmse: {rmse}")
    
    x_train = train.drop(['predicted_weight_g', 'Case', 'index'], axis=1)
    y_train = train['predicted_weight_g']
    
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    x_test['predicted_weight_g'] = y_pred
    submit = x_test[['DAT', 'predicted_weight_g']]
    submit['DAT'] = submit['DAT']+1
    all_target_list = sorted(glob.glob('./data/test_target/*.csv'))
    for idx, test_path in enumerate(all_target_list):
        submit_df = pd.read_csv(test_path)
        submit_df['predicted_weight_g'] = submit['predicted_weight_g'][idx*28:idx*28+28].values
        submit_df.to_csv(test_path, index=False)

In [15]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.linear_model import LinearRegression

xgb = XGBRegressor()
cat = CatBoostRegressor(verbose=100, early_stopping_rounds=50)
lgbm = LGBMRegressor()
linear = LinearRegression()

model_train(train, test, cat)

Learning rate set to 0.037231
0:	learn: 40.9796725	total: 168ms	remaining: 2m 48s
100:	learn: 7.8562335	total: 2.07s	remaining: 18.4s
200:	learn: 3.7618943	total: 4.01s	remaining: 16s
300:	learn: 2.4836647	total: 6.5s	remaining: 15.1s
400:	learn: 1.8555493	total: 9.14s	remaining: 13.7s
500:	learn: 1.4786309	total: 11.1s	remaining: 11s
600:	learn: 1.1688331	total: 13.2s	remaining: 8.76s
700:	learn: 0.9463365	total: 16.1s	remaining: 6.89s
800:	learn: 0.7689538	total: 18.4s	remaining: 4.57s
900:	learn: 0.6243814	total: 20.4s	remaining: 2.24s
999:	learn: 0.5182062	total: 22.7s	remaining: 0us
validation rmse: 6.464822524486032
Learning rate set to 0.039399
0:	learn: 40.4239842	total: 31.6ms	remaining: 31.6s
100:	learn: 7.1187196	total: 2.91s	remaining: 25.9s
200:	learn: 3.7356939	total: 4.93s	remaining: 19.6s
300:	learn: 2.6933190	total: 6.97s	remaining: 16.2s
400:	learn: 2.0530974	total: 9.89s	remaining: 14.8s
500:	learn: 1.6076572	total: 12.3s	remaining: 12.2s
600:	learn: 1.3139727	total:

C:\Users\kki96\AppData\Local\Temp\ipykernel_21400\2591069519.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['DAT'] = submit['DAT']+1


In [16]:
cat.feature_importances_

array([2.82935108e+01, 1.82067140e-01, 2.91266319e-02, 4.18348828e-02,
       3.39853068e-01, 5.87887157e-02, 2.18631086e-02, 2.97081649e-03,
       2.43489230e-02, 4.70375911e-02, 6.21179342e-03, 1.28247552e-02,
       4.29110808e-01, 9.72713392e-03, 5.98890772e-03, 9.65161451e-02,
       0.00000000e+00, 3.14596467e-02, 1.62314303e+01, 5.58896151e+00,
       3.14000423e+00, 1.40793843e-02, 3.06776861e+00, 1.84300585e-01,
       2.72134902e-01, 8.87946569e-02, 1.22610561e-01, 1.82348295e-01,
       6.37800194e-02, 1.74708444e-01, 1.83330105e-02, 3.45972731e-02,
       3.92330303e-01, 1.62900829e-01, 8.04896041e-02, 4.63753236e-01,
       1.63425267e+00, 5.24379855e-01, 5.68791017e-01, 1.15179463e-01,
       3.52095211e-01, 7.71658983e-02, 1.79785070e-01, 5.14086416e-02,
       4.98807672e-03, 2.16118771e-01, 3.59369356e-01, 5.12243386e-02,
       3.96234315e-02, 2.16827655e-01, 1.11361953e-01, 2.43726423e-01,
       2.08432293e-02, 3.94903150e-02, 3.90726662e-02, 4.06702359e-02,
      

In [17]:
df_imp = pd.DataFrame({'imp':cat.feature_importances_}, index = cat.feature_names_)
df_imp = df_imp[df_imp.imp > 0].sort_values('imp').copy()
df_imp

,imp
시간당청색광량2시,0.000051
일간누적청색광량12시,0.000086
시간당백색광량13시,0.000106
시간당백색광량22시,0.000107
일간누적청색광량2시,0.000123
...,...
월간누적총광량,3.067769
월간누적적색광량,3.140004
월간누적백색광량,5.588962
월간누적분무량,16.231430
